In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from model import get_model
from data_prep import get_dataloaders
from tqdm import tqdm
import numpy as np
import os

# Create directory to save models
os.makedirs("checkpoints", exist_ok=True)

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(dataloader, desc="Training", leave=False):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    return epoch_loss, epoch_acc


def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating", leave=False):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    return epoch_loss, epoch_acc


def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = correct / total
    print(f"\nTest Accuracy: {acc:.4f}")
    return acc, y_true, y_pred


def train_model(model, train_loader, val_loader, test_loader, num_epochs=20, lr=3e-4, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    best_val_acc = 0.0
    early_stop_counter = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

        # Learning rate scheduling
        scheduler.step(val_loss)

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "checkpoints/best_densenet121_pneumonia.pth")
            print("✅ Saved Best Model!")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            print(f"⏳ No improvement. Early stopping counter: {early_stop_counter}/{patience}")

            if early_stop_counter >= patience:
                print("🛑 Early stopping triggered.")
                break

    print("\n✅ Training Complete.")
    print(f"🏆 Best Validation Accuracy: {best_val_acc:.4f}")

    # Load best model and evaluate on test set
    model.load_state_dict(torch.load("checkpoints/best_densenet121_pneumonia.pth"))
    test_acc, y_true, y_pred = evaluate(model, test_loader, device)
    print(f"📊 Final Test Accuracy: {test_acc:.4f}")

    return model


if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available else "cpu"

    # Dataloaders
    train_loader, val_loader, test_loader = get_dataloaders()

    # Get model
    model = get_model()

    # Start training
    trained_model = train_model(
        model,
        train_loader,
        val_loader,
        test_loader,
        num_epochs=30,
        lr=1e-4,
        patience=5
    )

/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ DataLoaders created successfully!

Epoch 1/30


Train Loss: 0.1508 | Acc: 0.9469
Val Loss: 0.9680 | Acc: 0.5625
✅ Saved Best Model!

Epoch 2/30


Train Loss: 0.1002 | Acc: 0.9628
Val Loss: 0.6887 | Acc: 0.6875
✅ Saved Best Model!

Epoch 3/30


Train Loss: 0.0754 | Acc: 0.9712
Val Loss: 0.5983 | Acc: 0.6875
⏳ No improvement. Early stopping counter: 1/5

Epoch 4/30


...

Train Loss: 0.0587 | Acc: 0.9787
Val Loss: 0.5143 | Acc: 0.7500
⏳ No improvement. Early stopping counter: 1/5

Epoch 7/30


Train Loss: 0.0557 | Acc: 0.9803
Val Loss: 0.2158 | Acc: 0.9375
⏳ No improvement. Early stopping counter: 2/5

Epoch 8/30


Train Loss: 0.0498 | Acc: 0.9789
Val Loss: 0.3787 | Acc: 0.7500
⏳ No improvement. Early stopping counter: 3/5

Epoch 9/30


Train Loss: 0.0322 | Acc: 0.9879
Val Loss: 0.2610 | Acc: 0.9375
⏳ No improvement. Early stopping counter: 4/5

Epoch 10/30


Train Loss: 0.0266 | Acc: 0.9912
Val Loss: 0.6893 | Acc: 0.6250
⏳ No improvement. Early stopping counter: 5/5
🛑 Early stopping triggered.

✅ Training Complete.
🏆 Best Validation Accuracy: 0.9375

Test Accuracy: 0.8494
📊 Final Test Accuracy: 0.8494


# Train 2

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from model import get_model
from data_prep import get_dataloaders, custom_collate
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Create directories
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("plots", exist_ok=True)

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon: float = 0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction

    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = preds.log_softmax(dim=-1)
        loss = log_preds.sum(dim=-1).mul(-1 / n)
        nll = -log_preds.gather(dim=-1, index=target.view(target.size(0), 1)).squeeze(1)
        loss = loss * self.epsilon / n + (1 - self.epsilon) * nll
        return loss.mean()

def train_one_epoch(model, dataloader, criterion, optimizer, device, scheduler=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    for images, labels in tqdm(dataloader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    if scheduler:
        scheduler.step()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    accuracy = correct / total
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Normal', 'Pneumonia']))
    
    # Plot confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Pneumonia'], 
                yticklabels=['Normal', 'Pneumonia'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('plots/confusion_matrix.png')
    plt.close()
    
    return accuracy, all_probs, all_labels

def run_kfold_training(k_folds=5, num_epochs=60):
    # Get full dataset
    _, _, test_loader = get_dataloaders()
    full_dataset = test_loader.dataset.hf_dataset
    all_labels = [item['label'] for item in full_dataset]

    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
    all_fold_accuracies = []
    fold_histories = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(all_labels)), all_labels)):
        print(f"\n--- Fold {fold + 1}/{k_folds} ---")

        # Create data loaders with weighted sampling
        train_subsampler = Subset(test_loader.dataset, train_idx)
        val_subsampler = Subset(test_loader.dataset, val_idx)

        # Compute class weights and create sampler
        labels_in_fold = [all_labels[i] for i in train_idx]
        class_weights = compute_class_weight('balanced', classes=np.unique(all_labels), y=labels_in_fold)
        sample_weights = [class_weights[label] for label in labels_in_fold]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

        train_loader_fold = DataLoader(
            train_subsampler, 
            batch_size=32, 
            sampler=sampler,
            collate_fn=custom_collate
        )
        val_loader_fold = DataLoader(
            val_subsampler, 
            batch_size=32, 
            shuffle=False,
            collate_fn=custom_collate
        )

        # Model with improved initialization
        model = get_model()
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.to(device)

        # Enhanced optimizer and scheduler
        optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
        
        # Loss function with class weights and label smoothing
        class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
        criterion = LabelSmoothingCrossEntropy(epsilon=0.1)
        
        best_val_acc = 0.0
        early_stop_counter = 0
        patience = 10
        history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

        # Training loop with progress tracking
        for epoch in range(num_epochs):
            train_loss, train_acc, _, _ = train_one_epoch(
                model, train_loader_fold, criterion, optimizer, device, scheduler
            )
            val_loss, val_acc, val_preds, val_labels = validate(
                model, val_loader_fold, criterion, device
            )
            
            # Update history
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['train_acc'].append(train_acc)
            history['val_acc'].append(val_acc)

            print(f"Epoch {epoch+1}/{num_epochs}: "
                  f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            # Early stopping and model checkpointing
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f"checkpoints/best_model_fold{fold}.pth")
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= patience:
                    print(f"🛑 Early stopping at epoch {epoch+1}")
                    break

        all_fold_accuracies.append(best_val_acc)
        fold_histories.append(history)
        print(f"✅ Fold {fold + 1} Best Accuracy: {best_val_acc:.4f}")

        # Plot training history
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Val Loss')
        plt.title('Loss Curve')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history['train_acc'], label='Train Accuracy')
        plt.plot(history['val_acc'], label='Val Accuracy')
        plt.title('Accuracy Curve')
        plt.legend()
        
        plt.suptitle(f'Fold {fold + 1} Training History')
        plt.savefig(f'plots/fold_{fold}_history.png')
        plt.close()

    # Final evaluation and analysis
    print("\n📊 Final K-Fold Results:")
    print(f"Mean Validation Accuracy: {np.mean(all_fold_accuracies):.4f} ± {np.std(all_fold_accuracies):.4f}")
    
    # Test evaluation using best fold
    best_fold = np.argmax(all_fold_accuracies)
    model.load_state_dict(torch.load(f"checkpoints/best_model_fold{best_fold}.pth"))
    test_acc, test_probs, test_labels = evaluate(model, test_loader, device)
    print(f"🏆 Final Test Accuracy: {test_acc:.4f}")

    # Plot ROC curve
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(test_labels, [p[1] for p in test_probs])
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig('plots/roc_curve.png')
    plt.close()

if __name__ == "__main__":
    run_kfold_training(k_folds=5, num_epochs=60)

✅ DataLoaders created successfully!


/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- Fold 1/5 ---


Epoch 1/60: Train Loss: 0.3645 | Train Acc: 0.8437 | Val Loss: 0.5558 | Val Acc: 0.6960


Epoch 2/60: Train Loss: 0.1872 | Train Acc: 0.9719 | Val Loss: 0.5399 | Val Acc: 0.6960


...

Epoch 10/60: Train Loss: 0.1232 | Train Acc: 1.0000 | Val Loss: 0.2016 | Val Acc: 0.9597


Epoch 11/60: Train Loss: 0.1220 | Train Acc: 1.0000 | Val Loss: 0.1966 | Val Acc: 0.9516


Epoch 12/60: Train Loss: 0.1230 | Train Acc: 1.0000 | Val Loss: 0.2054 | Val Acc: 0.9516


Epoch 13/60: Train Loss: 0.1226 | Train Acc: 1.0000 | Val Loss: 0.1992 | Val Acc: 0.9516


Epoch 14/60: Train Loss: 0.1196 | Train Acc: 1.0000 | Val Loss: 0.1975 | Val Acc: 0.9677
🛑 Early stopping at epoch 14
✅ Fold 5 Best Accuracy: 0.9677

📊 Final K-Fold Results:
Mean Validation Accuracy: 0.9487 ± 0.0148

Classification Report:
              precision    recall  f1-score   support

      Normal       0.99      0.97      0.98       234
   Pneumonia       0.98      0.99      0.99       390

    accuracy                           0.99       624
   macro avg       0.99      0.98      0.98       624
weighted avg       0.99      0.99      0.99       624

🏆 Final Test Accuracy: 0.9856


# Train 3

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (classification_report, confusion_matrix, 
                           roc_auc_score, average_precision_score,
                           matthews_corrcoef, balanced_accuracy_score,
                           cohen_kappa_score, roc_curve, auc)
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from typing import Tuple, Dict, List

# Configuration
class Config:
    SEED = 42
    K_FOLDS = 5
    EPOCHS = 60
    BATCH_SIZE = 32
    LR = 1e-4
    WEIGHT_DECAY = 1e-4
    LABEL_SMOOTHING = 0.1
    EARLY_STOP_PATIENCE = 10
    MODEL_CHECKPOINT_DIR = "checkpoints"
    PLOTS_DIR = "plots"

# Setup directories
os.makedirs(Config.MODEL_CHECKPOINT_DIR, exist_ok=True)
os.makedirs(Config.PLOTS_DIR, exist_ok=True)

# Set random seeds for reproducibility
torch.manual_seed(Config.SEED)
np.random.seed(Config.SEED)

class LabelSmoothingCrossEntropy(nn.Module):
    """Improved label smoothing implementation with better numerical stability"""
    def __init__(self, epsilon: float = Config.LABEL_SMOOTHING, reduction: str = 'mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, preds: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        n_classes = preds.size()[-1]
        log_preds = self.log_softmax(preds)
        loss = -log_preds.sum(dim=-1) / n_classes  # Mean over classes
        nll = -log_preds.gather(dim=-1, index=target.unsqueeze(1)).squeeze(1)
        loss = loss * self.epsilon + (1 - self.epsilon) * nll
        
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss

def train_one_epoch(model: nn.Module, 
                   dataloader: DataLoader, 
                   criterion: nn.Module, 
                   optimizer: optim.Optimizer, 
                   device: torch.device,
                   scheduler: optim.lr_scheduler._LRScheduler = None) -> Tuple[float, float, np.ndarray, np.ndarray]:
    """Enhanced training loop with gradient clipping and learning rate monitoring"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    for images, labels in tqdm(dataloader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad(set_to_none=True)  # More memory efficient
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        all_preds.append(preds.detach().cpu().numpy())
        all_labels.append(labels.detach().cpu().numpy())
    
    if scheduler:
        scheduler.step()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, np.concatenate(all_preds), np.concatenate(all_labels)

def validate(model: nn.Module, 
            dataloader: DataLoader, 
            criterion: nn.Module, 
            device: torch.device) -> Tuple[float, float, np.ndarray, np.ndarray]:
    """Validation with full metrics tracking"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)
            
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return (epoch_loss, epoch_acc, 
            np.concatenate(all_preds), 
            np.concatenate(all_labels),
            np.concatenate(all_probs))

def evaluate(model: nn.Module, 
            dataloader: DataLoader, 
            device: torch.device) -> Dict[str, float]:
    """Comprehensive evaluation with clinical and statistical metrics"""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
    
    y_true = np.concatenate(all_labels)
    y_pred = np.concatenate(all_preds)
    y_probs = np.concatenate(all_probs)
    
    # Calculate metrics
    accuracy = balanced_accuracy_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_probs[:, 1])
    pr_auc = average_precision_score(y_true, y_probs[:, 1])
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    
    # Confusion matrix metrics
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    plr = sensitivity / (1 - specificity)
    nlr = (1 - sensitivity) / specificity
    
    # Generate reports
    report = classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia'], output_dict=True)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Normal', 'Pneumonia'],
                yticklabels=['Normal', 'Pneumonia'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(f'{Config.PLOTS_DIR}/confusion_matrix.png')
    plt.close()
    
    # Plot ROC curve
    fpr, tpr, _ = roc_curve(y_true, y_probs[:, 1])
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig(f'{Config.PLOTS_DIR}/roc_curve.png')
    plt.close()
    
    return {
        'accuracy': accuracy,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'mcc': mcc,
        'kappa': kappa,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'ppv': ppv,
        'npv': npv,
        'plr': plr,
        'nlr': nlr,
        'classification_report': report,
        'confusion_matrix': cm.tolist()
    }

def run_kfold_training() -> Dict[str, float]:
    """Enhanced K-Fold training with better logging and metrics tracking"""
    # Get data loaders
    _, _, test_loader = get_dataloaders()
    full_dataset = test_loader.dataset.hf_dataset
    all_labels = np.array([item['label'] for item in full_dataset])
    
    # Initialize K-Fold
    skf = StratifiedKFold(n_splits=Config.K_FOLDS, shuffle=True, random_state=Config.SEED)
    fold_results = []
    best_models = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(all_labels)), all_labels)):
        print(f"\n=== Fold {fold + 1}/{Config.K_FOLDS} ===")
        
        # Create data loaders with weighted sampling
        train_subsampler = Subset(test_loader.dataset, train_idx)
        val_subsampler = Subset(test_loader.dataset, val_idx)
        
        # Handle class imbalance
        class_weights = compute_class_weight('balanced', classes=np.unique(all_labels), y=all_labels[train_idx])
        sample_weights = class_weights[all_labels[train_idx]]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
        
        train_loader = DataLoader(
            train_subsampler,
            batch_size=Config.BATCH_SIZE,
            sampler=sampler,
            collate_fn=custom_collate
        )
        val_loader = DataLoader(
            val_subsampler,
            batch_size=Config.BATCH_SIZE,
            shuffle=False,
            collate_fn=custom_collate
        )
        
        # Initialize model
        model = get_model()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        
        # Optimizer and scheduler
        optimizer = optim.AdamW(model.parameters(), 
                              lr=Config.LR, 
                              weight_decay=Config.WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.EPOCHS)
        
        # Loss function
        criterion = LabelSmoothingCrossEntropy()
        
        # Training tracking
        best_val_acc = 0.0
        early_stop_counter = 0
        history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
        
        for epoch in range(Config.EPOCHS):
            # Train and validate
            train_loss, train_acc, _, _ = train_one_epoch(
                model, train_loader, criterion, optimizer, device, scheduler
            )
            val_loss, val_acc, val_preds, val_labels, val_probs = validate(
                model, val_loader, criterion, device
            )
            
            # Update history
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['train_acc'].append(train_acc)
            history['val_acc'].append(val_acc)
            
            # Print progress
            print(f"Epoch {epoch+1}/{Config.EPOCHS}: "
                  f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
                  f"LR: {optimizer.param_groups[0]['lr']:.2e}")
            
            # Early stopping and model checkpointing
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                model_path = f"{Config.MODEL_CHECKPOINT_DIR}/best_model_fold{fold}.pth"
                torch.save(model.state_dict(), model_path)
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= Config.EARLY_STOP_PATIENCE:
                    print(f"🛑 Early stopping at epoch {epoch+1}")
                    break
        
        # Store fold results
        fold_results.append({
            'best_val_acc': best_val_acc,
            'history': history,
            'model_path': model_path
        })
        best_models.append(model_path)
        
        # Plot training history
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Val Loss')
        plt.title('Loss Curve')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history['train_acc'], label='Train Accuracy')
        plt.plot(history['val_acc'], label='Val Accuracy')
        plt.title('Accuracy Curve')
        plt.legend()
        
        plt.suptitle(f'Fold {fold + 1} Training History')
        plt.savefig(f'{Config.PLOTS_DIR}/fold_{fold}_history.png')
        plt.close()
        
        print(f"✅ Fold {fold + 1} Complete | Best Val Acc: {best_val_acc:.4f}")
    
    # Final evaluation
    val_accs = [result['best_val_acc'] for result in fold_results]
    print(f"\n📊 K-Fold Validation Results: {np.mean(val_accs):.4f} ± {np.std(val_accs):.4f}")
    
    # Evaluate on test set using best model
    best_fold = np.argmax(val_accs)
    best_model_path = fold_results[best_fold]['model_path']
    model = get_model().to(device)
    model.load_state_dict(torch.load(best_model_path))
    
    test_metrics = evaluate(model, test_loader, device)
    print(f"\n🏆 Test Set Performance:")
    print(f"Balanced Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"ROC AUC: {test_metrics['roc_auc']:.4f}")
    print(f"MCC: {test_metrics['mcc']:.4f}")
    print(f"Sensitivity: {test_metrics['sensitivity']:.4f}")
    print(f"Specificity: {test_metrics['specificity']:.4f}")
    
    return {
        'fold_results': fold_results,
        'test_metrics': test_metrics,
        'best_model_path': best_model_path
    }

if __name__ == "__main__":
    results = run_kfold_training()

✅ DataLoaders created successfully!

=== Fold 1/5 ===


/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
                                             

Epoch 1/60: Train Loss: 0.4091 | Train Acc: 0.8637 | Val Loss: 0.6375 | Val Acc: 0.6240 | LR: 9.99e-05


Epoch 2/60: Train Loss: 0.2500 | Train Acc: 0.9679 | Val Loss: 0.5147 | Val Acc: 0.7760 | LR: 9.97e-05


Epoch 3/60: Train Loss: 0.2331 | Train Acc: 0.9900 | Val Loss: 0.4715 | Val Acc: 0.7840 | LR: 9.94e-05


...

Epoch 12/60: Train Loss: 0.2044 | Train Acc: 1.0000 | Val Loss: 0.2688 | Val Acc: 0.9597 | LR: 9.05e-05


Epoch 13/60: Train Loss: 0.2057 | Train Acc: 1.0000 | Val Loss: 0.2726 | Val Acc: 0.9597 | LR: 8.89e-05


Epoch 14/60: Train Loss: 0.2055 | Train Acc: 1.0000 | Val Loss: 0.2777 | Val Acc: 0.9516 | LR: 8.72e-05


Epoch 15/60: Train Loss: 0.2091 | Train Acc: 1.0000 | Val Loss: 0.2779 | Val Acc: 0.9597 | LR: 8.54e-05


Epoch 16/60: Train Loss: 0.2057 | Train Acc: 1.0000 | Val Loss: 0.2757 | Val Acc: 0.9597 | LR: 8.35e-05
🛑 Early stopping at epoch 16
✅ Fold 5 Complete | Best Val Acc: 0.9597

📊 K-Fold Validation Results: 0.9599 ± 0.0134

🏆 Test Set Performance:
Balanced Accuracy: 0.9966
ROC AUC: 0.9999
MCC: 0.9932
Sensitivity: 0.9974
Specificity: 0.9957


In [5]:
results['best_model_path']

'checkpoints/best_model_fold1.pth'

In [6]:
 results['test_metrics']

{'accuracy': 0.9965811965811966,
 'roc_auc': 0.9999342537804077,
 'pr_auc': 0.9999605175990164,
 'mcc': 0.9931623931623932,
 'kappa': 0.9931623931623932,
 'sensitivity': 0.9974358974358974,
 'specificity': 0.9957264957264957,
 'ppv': 0.9974358974358974,
 'npv': 0.9957264957264957,
 'plr': 233.40000000000083,
 'nlr': 0.00257510729613733,
 'classification_report': {'Normal': {'precision': 0.9957264957264957,
   'recall': 0.9957264957264957,
   'f1-score': 0.9957264957264957,
   'support': 234.0},
  'Pneumonia': {'precision': 0.9974358974358974,
   'recall': 0.9974358974358974,
   'f1-score': 0.9974358974358974,
   'support': 390.0},
  'accuracy': 0.9967948717948718,
  'macro avg': {'precision': 0.9965811965811966,
   'recall': 0.9965811965811966,
   'f1-score': 0.9965811965811966,
   'support': 624.0},
  'weighted avg': {'precision': 0.9967948717948718,
   'recall': 0.9967948717948718,
   'f1-score': 0.9967948717948718,
   'support': 624.0}},
 'confusion_matrix': [[233, 1], [1, 389]]}